In [1]:
import pandas as pd
import subprocess
import numpy as np
import pandas_profiling

from datetime import datetime

In [2]:
ruta_archivo = 'DatosOriginales.csv' #Ruta al csv generado con OpenRefine
ruta_salida = 'DatosPreprocesados.csv' #Ruta del archivo csv de salida
weka = "C:/Program Files/Weka-3-8/weka.jar" #Ruta al jar de weka
arff_path = ruta_salida.replace('.csv','.arff') #Ruta al arff de salida.

# Leer el archivo

In [3]:
df = pd.read_csv(ruta_archivo)

print(len(df))

36769


# DF ANTES

In [4]:
df.head()

,Fecha,Departamento,Municipio,Día,Hora,Barrio,Zona,Clase de sitio,Arma empleada,Móvil Agresor,...,Estado civil,País de nacimiento,Clase de empleado,Profesión,Escolaridad,Código DANE,CLASE,MARCA,LINEA,Cantidad
0,01/01/2017 12:00:00 AM,AMAZONAS,LETICIA (CT),Domingo,12/31/1899 08:00:00 AM,JARDIN,URBANA,CASAS DE HABITACION,CONTUNDENTES,A PIE,...,UNION LIBRE,COLOMBIA,INDEPENDIENTE,-,TECNICO,91001000,CELULAR,SAMSUNG,J700,1
1,01/01/2017 12:00:00 AM,ANTIOQUIA,BELLO,Domingo,12/31/1899 07:00:00 PM,CIUDADELA DEL NORTE,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,...,SOLTERO,COLOMBIA,EMPLEADO PARTICULAR,-,PRIMARIA,5088000,CELULAR,SAMSUNG,GALAXY J,1
2,01/01/2017 12:00:00 AM,ANTIOQUIA,BELLO,Domingo,12/31/1899 12:00:00 AM,VDA. SAN FELIX,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,...,SOLTERO,COLOMBIA,EMPLEADO PARTICULAR,-,SECUNDARIA,5088000,CELULAR,LENOVO,LINEA STANDARD,1
3,01/01/2017 12:00:00 AM,ANTIOQUIA,BELLO,Domingo,12/31/1899 09:45:00 PM,SANTA RITA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,PASAJERO MOTOCICLETA,...,SOLTERO,COLOMBIA,INDEPENDIENTE,-,SECUNDARIA,5088000,CELULAR,SAMSUNG,GALAXY J,1
4,01/01/2017 12:00:00 AM,ANTIOQUIA,BELLO,Domingo,12/31/1899 10:35:00 AM,VDA. SAN FELIX,RURAL,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,PASAJERO MOTOCICLETA,...,CASADO,COLOMBIA,INDEPENDIENTE,-,SECUNDARIA,5088000,CELULAR,IPHONE,5S,1


# Eliminar filas con cantidad mayor a 3

In [5]:
print(len(df))
indices_cantidad_invalida = df['Cantidad'] > 3
df = df.drop(df.index[indices_cantidad_invalida])
print(len(df))

36769
36566


# Eliminar columnas

In [6]:
df.columns

Index(['Fecha', 'Departamento', 'Municipio', 'Día', 'Hora', 'Barrio', 'Zona',
       'Clase de sitio', 'Arma empleada', 'Móvil Agresor', 'Móvil Victima',
       'Edad', 'Sexo', 'Estado civil', 'País de nacimiento',
       'Clase de empleado', 'Profesión', 'Escolaridad', 'Código DANE', 'CLASE',
       'MARCA', 'LINEA', 'Cantidad'],
      dtype='object')

In [7]:
del df['Código DANE']
del df['CLASE']
del df['Estado civil']
del df['LINEA']
del df['Cantidad']
del df['Escolaridad']

In [8]:
df.columns

Index(['Fecha', 'Departamento', 'Municipio', 'Día', 'Hora', 'Barrio', 'Zona',
       'Clase de sitio', 'Arma empleada', 'Móvil Agresor', 'Móvil Victima',
       'Edad', 'Sexo', 'País de nacimiento', 'Clase de empleado', 'Profesión',
       'MARCA'],
      dtype='object')

# Renombrar columnas

In [9]:
df.columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'DIA', 'HORA', 'BARRIO', 'ZONA', 'SITIO', 'ARMA', 
              'MOVILAGRESOR', 'MOVILVICTIMA', 'EDAD', 'SEXO', 'NACIONALIDAD', 'CLASEEMPLEADO', 'PROFESION', 'MARCA']

In [10]:
df.columns

Index(['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'DIA', 'HORA', 'BARRIO', 'ZONA',
       'SITIO', 'ARMA', 'MOVILAGRESOR', 'MOVILVICTIMA', 'EDAD', 'SEXO',
       'NACIONALIDAD', 'CLASEEMPLEADO', 'PROFESION', 'MARCA'],
      dtype='object')

In [11]:
columnas_str = ['DEPARTAMENTO', 'MUNICIPIO', 'DIA', 'BARRIO', 'ZONA', 'SITIO', 'ARMA', 'MOVILAGRESOR', 
                'MOVILVICTIMA', 'SEXO', 'NACIONALIDAD', 'CLASEEMPLEADO', 'PROFESION', 'MARCA']

df[columnas_str] = df[columnas_str].astype(str) 

# Cambiar formato del texto
    
    Poner todo en mayuscula
    Quitar las tildes
    Cambiar las Ñ por NH

In [12]:
for col_str in columnas_str:
    df[col_str] = df[col_str].str.upper()

In [13]:
caracter_anterior = ['Á', 'É', 'Í', 'Ó', 'Ú', 'Ü', 'Ñ']
caracter_cambiado = ['A', 'E', 'I', 'O', 'U', 'U', 'NH']

df = df.replace(caracter_anterior, caracter_cambiado, regex=True)

# Combinar columnas de profesión y clase de empleado

    Se asigna el valor de la columna profesión con clase de empleado donde no haya profesión se pone ese valor.

In [14]:
indices_profesion_vacia = np.where(df['PROFESION'] == '-')[0]
print(len(indices_profesion_vacia))

32035


In [15]:
col_loc_profesion = df.columns.get_loc('PROFESION')
col_loc_clasedeempleado = df.columns.get_loc('CLASEEMPLEADO')

for ipv in indices_profesion_vacia:
    df.iloc[ipv, col_loc_profesion] = df.iloc[ipv, col_loc_clasedeempleado]

In [16]:
indices_profesion_vacia = np.where(df['PROFESION'] == '-')[0]
print(len(indices_profesion_vacia))

472


In [17]:
del df['CLASEEMPLEADO']
del df['PROFESION']

# Arreglar columnas de fecha y hora

In [18]:
col_loc_fecha = df.columns.get_loc('FECHA')
col_loc_hora = df.columns.get_loc('HORA')

for i in range(len(df)):
    fecha_antes = df.iloc[i, col_loc_fecha]
    fecha_despues = fecha_antes.split(' ')[0]
    df.iloc[i, col_loc_fecha] = fecha_despues
    
    hora_antes = df.iloc[i, col_loc_hora]
    hora_despues = hora_antes.split(' ', 1)[-1]
    df.iloc[i, col_loc_hora] = hora_despues

# Eliminar marcas no comunes de celulares

In [19]:
df['MARCA'].value_counts()

SAMSUNG             13175
HUAWEI               6190
MOTOROLA             3565
NO REPORTADO         2590
IPHONE               2092
ALCATEL              2034
LG                   1416
LENOVO                839
NOKIA                 737
AVVIO                 698
SONY                  622
ASUS                  368
ZTE                   357
LANIX                 260
AZUMI                 255
BLU                   254
-                     241
APPLE                 155
HTC                    98
BLACKBERRY             83
IPRO                   77
AMBA                   65
SENDTEL                59
MICROSOFT              39
IMOBILE                36
MOVIC                  29
BELLSOUTH              19
BENQ                   14
CELLACOM               13
QUIX                   12
                    ...  
VERYKOOL               10
GARMIN                  9
ERICSSON                9
VODAFONE                9
TOSHIBA                 8
KYOCERA                 8
COMPAL                  7
GIGABYTE    

In [20]:
col_loc_marca = df.columns.get_loc('MARCA')

indices_marca_noreportada = np.where(df['MARCA'] == 'NO REPORTADO')[0]
for imn in indices_marca_noreportada:
    df.iloc[imn, col_loc_marca] = '-'
    
indices_marca_iphone = np.where(df['MARCA'] == 'IPHONE')[0]
for imi in indices_marca_iphone:
    df.iloc[imi, col_loc_marca] = 'APPLE'

In [21]:
marcas_counts = df['MARCA'].value_counts() >= 100
marcas_destacadas = marcas_counts[np.where(marcas_counts == True)[0]]
marcas_destacadas = np.array(marcas_destacadas.keys())
marcas_otras = marcas_counts[np.where(marcas_counts == False)[0]]
marcas_otras = np.array(marcas_otras.keys())

for i in range(len(df)):    
    m = df.iloc[i, col_loc_marca]
    if m in marcas_otras:
        df.iloc[i, col_loc_marca] = 'OTRA'

In [22]:
df['MARCA'].value_counts()

SAMSUNG     13175
HUAWEI       6190
MOTOROLA     3565
-            2831
APPLE        2247
ALCATEL      2034
LG           1416
LENOVO        839
NOKIA         737
OTRA          718
AVVIO         698
SONY          622
ASUS          368
ZTE           357
LANIX         260
AZUMI         255
BLU           254
Name: MARCA, dtype: int64

# Categorizamos las Edades

    Infante: 0 - 7

    Nino: 8 - 12
    
    Adolescente: 13 - 17
    
    Adulto Joven: 18 - 30
    
    Adulto: 31 - 45
    
    Adulto Mayor: 46 - 60
    
    Anciano: 60 - 110

In [23]:
col_loc_edad = df.columns.get_loc('EDAD')

indices_edad_noreportada = np.where(df['EDAD'] == '-')[0]
for ien in indices_edad_noreportada:
    df.iloc[ien, col_loc_edad] = -1
    
df['EDAD'] = df['EDAD'].apply(pd.to_numeric)

In [24]:
df_edad_categorizada = pd.cut(df['EDAD'], bins=[-1, 0, 7, 12, 17, 30, 45, 60, 110], include_lowest=True, 
                              labels=['OTRA','INFANTE', 'NINHO', 'ADOLESCENTE', 'ADULTOJOVEN', 'ADULTO', 
                                      'ADULTOMAYOR', 'ANCIANO'])

df['EDAD'] = df_edad_categorizada

# Categorizamos las fechas en meses y horas por tiempos del día

    Madrugada: 12:01 AM - 6:00 AM

    Movilizacion Matutina: 6:01 AM - 8:30 AM
    
    Manhana: 8:31 AM - 12:00 PM
    
    Almuerzo: 12:01 PM - 2:00 PM
    
    Tarde: 2:01 PM - 5:00 PM
    
    Movilizacion Vespertina: 5:01 PM - 8:00 PM
    
    Noche: 8:01 PM - 12:00 AM 

In [25]:
meses = { 
    1 : 'ENERO',
    2 : 'FEBRERO',
    3 : 'MARZO',
    4 : 'ABRIL',
    5 : 'MAYO',
    6 : 'JUNIO',
    7 : 'JULIO',
    8 : 'AGOSTO',
    9 : 'SEPTIEMBRE',
    10 : 'OCTUBRE',
    11 : 'NOVIEMBRE',
    12 : 'DICIEMBRE'   
}

horas_limite = {
    'madrugada_inicio': '1900-01-01 12:01 AM',
    'madrugada_final' : '1900-01-01 06:00 AM',
    'mov_matutina_inicio': '1900-01-01 06:01 AM',
    'mov_matutina_inicio': '1900-01-01 08:30 AM',
    'manhana_inicio': '1900-01-01 08:31 AM',
    'manhana_final': '1900-01-01 12:00 PM',
    'almuerzo_inicio': '1900-01-01 12:01 PM',
    'almuerzo_final': '1900-01-01 02:00 PM',
    'tarde_inicio': '1900-01-01 02:01 PM',
    'tarde_final': '1900-01-01 05:00 PM',
    'mov_vespertina_inicio': '1900-01-01 05:01 PM',
    'mov_vespertina_final': '1900-01-01 08:00 PM',
    'noche_inicio': '1900-01-01 08:01 PM',
    'noche_final': '1900-01-02 12:00 AM' # notese que aqui cambie al siguiente dia
}

In [26]:
def category_by_hour(hora):
    '''Da la categoría (de hora) del día dependiendo de la hora que mandemos como parámetro'''
    formato_hora_limite = '%Y-%m-%d %I:%M %p'    
    madrugada = datetime.strptime( horas_limite['madrugada_inicio'], formato_hora_limite)
    mov_matutina = datetime.strptime( horas_limite['mov_matutina_inicio'], formato_hora_limite)
    manhana = datetime.strptime( horas_limite['manhana_inicio'], formato_hora_limite)
    almuerzo = datetime.strptime( horas_limite['almuerzo_inicio'], formato_hora_limite)
    tarde = datetime.strptime( horas_limite['tarde_inicio'], formato_hora_limite)
    mov_vespertina = datetime.strptime( horas_limite['mov_vespertina_inicio'], formato_hora_limite)
    noche = datetime.strptime( horas_limite['noche_inicio'], formato_hora_limite)
    
    categoria = 'no-definida'
    if hora >= madrugada and hora < mov_matutina:
        categoria = 'MADRUGADA'
    elif hora >= mov_matutina and hora < manhana:
        categoria = 'MOVILIZACIONMATUTINA'
    elif hora >= manhana and hora < almuerzo:
        categoria = 'MANHANA'
    elif hora >= almuerzo and hora < tarde:
        categoria = 'ALMUERZO'
    elif hora >= tarde and hora < mov_vespertina:
        categoria = 'TARDE'
    elif hora >= mov_vespertina and hora < noche:
        categoria = 'MOVILIZACIONVESPERTINA'
    else:
        categoria  = 'NOCHE'
    
    return categoria

In [27]:
formato_fecha = '%m/%d/%Y'
formato_hora = '%I:%M:%S %p'

col_loc_fecha = df.columns.get_loc('FECHA')
col_loc_hora = df.columns.get_loc('HORA')

for i in range(len(df)):
    fecha_string = df.iloc[i, col_loc_fecha]
    fecha = datetime.strptime(fecha_string, formato_fecha)
    df.iloc[i, col_loc_fecha] = meses[fecha.month]
    
    hora_string = df.iloc[i, col_loc_hora]
    hora  = datetime.strptime(hora_string, formato_hora)
    df.iloc[i, col_loc_hora] = category_by_hour(hora)

# Generalizar arma

In [28]:
df['ARMA'].value_counts()

SIN EMPLEO DE ARMAS            13437
ARMA BLANCA / CORTOPUNZANTE    11482
ARMA DE FUEGO                   8148
CONTUNDENTES                    2993
ESCOPOLAMINA                     309
LLAVE MAESTRA                    123
PALANCAS                          54
PERRO                              9
-                                  6
JERINGA                            4
CORTANTES                          1
Name: ARMA, dtype: int64

In [29]:
col_loc_arma = df.columns.get_loc('ARMA')

indices_arma_jeringa = np.where(df['ARMA'] == 'JERINGA')[0]
indices_arma_cortantes = np.where(df['ARMA'] == 'CORTANTES')[0]

indices_arma_cortopunzantes = np.concatenate((indices_arma_jeringa, indices_arma_cortantes))

for iac in indices_arma_cortopunzantes:
    df.iloc[iac, col_loc_arma] = 'ARMA BLANCA / CORTOPUNZANTE'

In [30]:
df['ARMA'].value_counts()

SIN EMPLEO DE ARMAS            13437
ARMA BLANCA / CORTOPUNZANTE    11487
ARMA DE FUEGO                   8148
CONTUNDENTES                    2993
ESCOPOLAMINA                     309
LLAVE MAESTRA                    123
PALANCAS                          54
PERRO                              9
-                                  6
Name: ARMA, dtype: int64

# Generalizar movil agresor

In [31]:
df['MOVILAGRESOR'].value_counts()

A PIE                    25124
PASAJERO MOTOCICLETA      5692
CONDUCTOR MOTOCICLETA     2566
PASAJERO BUS              1401
BICICLETA                  728
PASAJERO TAXI              302
CONDUCTOR TAXI             222
CONDUCTOR VEHICULO         180
PASAJERO VEHICULO          179
PASAJERO METRO             132
-                           20
CONDUCTOR BUS               14
PASAJERO BARCO               4
PASAJERO AERONAVE            2
Name: MOVILAGRESOR, dtype: int64

In [32]:
col_loc_movilagresor = df.columns.get_loc('MOVILAGRESOR')

indices_movilagresor_motocicleta = np.concatenate( (np.where(df['MOVILAGRESOR'] == 'PASAJERO MOTOCICLETA')[0], 
                                                    np.where(df['MOVILAGRESOR'] == 'CONDUCTOR MOTOCICLETA')[0]) )
for imam in indices_movilagresor_motocicleta:
    df.iloc[imam, col_loc_movilagresor] = 'MOTOCICLETA'
    
indices_movilagresor_vehiculo = np.concatenate( (np.where(df['MOVILAGRESOR'] == 'PASAJERO VEHICULO')[0], 
                                                    np.where(df['MOVILAGRESOR'] == 'CONDUCTOR VEHICULO')[0]) )
for imav in indices_movilagresor_vehiculo:
    df.iloc[imav, col_loc_movilagresor] = 'VEHICULO'
    
indices_movilagresor_transportep = np.concatenate( (np.where(df['MOVILAGRESOR'] == 'PASAJERO BUS')[0], 
                                                    np.where(df['MOVILAGRESOR'] == 'PASAJERO TAXI')[0],
                                                    np.where(df['MOVILAGRESOR'] == 'CONDUCTOR TAXI')[0],
                                                    np.where(df['MOVILAGRESOR'] == 'PASAJERO METRO')[0],
                                                    np.where(df['MOVILAGRESOR'] == 'CONDUCTOR BUS')[0],) )
for imat in indices_movilagresor_transportep:
    df.iloc[imat, col_loc_movilagresor] = 'TRANSPORTE PUBLICO'

In [33]:
df['MOVILAGRESOR'].value_counts()

A PIE                 25124
MOTOCICLETA            8258
TRANSPORTE PUBLICO     2071
BICICLETA               728
VEHICULO                359
-                        20
PASAJERO BARCO            4
PASAJERO AERONAVE         2
Name: MOVILAGRESOR, dtype: int64

# Generalizar movil victima

In [34]:
df['MOVILVICTIMA'].value_counts()

A PIE                    30926
PASAJERO BUS              1572
CONDUCTOR VEHICULO        1281
CONDUCTOR MOTOCICLETA     1244
PASAJERO TAXI              319
CONDUCTOR TAXI             297
PASAJERO MOTOCICLETA       290
PASAJERO VEHICULO          216
BICICLETA                  196
PASAJERO METRO             129
CONDUCTOR BUS               63
-                           27
PASAJERO BARCO               3
PASAJERO AERONAVE            3
Name: MOVILVICTIMA, dtype: int64

In [35]:
col_loc_movilvictima = df.columns.get_loc('MOVILVICTIMA')

indices_movilvictima_motocicleta = np.concatenate( (np.where(df['MOVILVICTIMA'] == 'PASAJERO MOTOCICLETA')[0], 
                                                    np.where(df['MOVILVICTIMA'] == 'CONDUCTOR MOTOCICLETA')[0]) )
for imvm in indices_movilvictima_motocicleta:
    df.iloc[imvm, col_loc_movilvictima] = 'MOTOCICLETA'
    
indices_movilvictima_vehiculo = np.concatenate( (np.where(df['MOVILVICTIMA'] == 'PASAJERO VEHICULO')[0], 
                                                 np.where(df['MOVILVICTIMA'] == 'CONDUCTOR VEHICULO')[0]) )
for imvv in indices_movilvictima_vehiculo:
    df.iloc[imvv, col_loc_movilvictima] = 'VEHICULO'
    
indices_movilvictima_transportep = np.concatenate( (np.where(df['MOVILVICTIMA'] == 'PASAJERO BUS')[0], 
                                                    np.where(df['MOVILVICTIMA'] == 'PASAJERO TAXI')[0],
                                                    np.where(df['MOVILVICTIMA'] == 'CONDUCTOR TAXI')[0],
                                                    np.where(df['MOVILVICTIMA'] == 'PASAJERO METRO')[0],
                                                    np.where(df['MOVILVICTIMA'] == 'CONDUCTOR BUS')[0],) )
for imvt in indices_movilvictima_transportep:
    df.iloc[imvt, col_loc_movilvictima] = 'TRANSPORTE PUBLICO'

In [36]:
df['MOVILVICTIMA'].value_counts()

A PIE                 30926
TRANSPORTE PUBLICO     2380
MOTOCICLETA            1534
VEHICULO               1497
BICICLETA               196
-                        27
PASAJERO BARCO            3
PASAJERO AERONAVE         3
Name: MOVILVICTIMA, dtype: int64

# Categorizar nacionalidad por continente

In [37]:
df['NACIONALIDAD'].value_counts()

COLOMBIA                35548
-                         556
VENEZUELA                  87
ESTADOS UNIDOS             57
FRANCIA                    30
HOLANDA                    23
AUSTRALIA                  21
ECUADOR                    21
ESPANHA                    20
BRASIL                     20
ARGENTINA                  19
MEXICO                     18
ALEMANIA                   18
ITALIA                     13
REINO UNIDO                12
CANADA                     11
PERU                        8
PANAMA                      8
CHINA                       6
INGLATERRA                  6
ISRAEL                      6
RUSIA                       5
CHILE                       5
COREA                       5
FINLANDIA                   4
SUIZA                       4
SUECIA                      4
IRLANDA                     3
REPUBLICA DOMINICANA        3
UCRANIA                     2
URUGUAY                     2
NORUEGA                     2
DINAMARCA                   1
JAPON     

In [38]:
# es temporal que sea manual

paises_americanorte = ['ESTADOS UNIDOS', 'CANADA', 'MEXICO']
paises_americacentro = ['PANAMA', 'REPUBLICA DOMINICANA', 'HONDURAS', 'COSTA RICA', 'BAHAMAS', 'CUBA']
paises_americasur = ['VENEZUELA', 'ECUADOR', 'BRASIL', 'ARGENTINA', 'PERU', 'CHILE', 'URUGUAY', 'PARAGUAY', 'BOLIVIA']
paises_africa = ['KENYA']
paises_europa = ['FRANCIA', 'HOLANDA', 'ESPANHA', 'ALEMANIA', 'ITALIA', 'REINO UNIDO', 'INGLATERRA', 'FINLANDIA', 
                 'SUIZA', 'SUECIA', 'IRLANDA', 'NORUEGA', 'UCRANIA', 'BELGICA', 'PORTUGAL', 'GRECIA', 'DINAMARCA',
                 'POLONIA', 'REPUBLICA CHECA', 'AUSTRIA', 'ESTONIA']
paises_asia = ['JORDANIA', 'ISRAEL', 'CHINA', 'RUSIA', 'COREA', 'AFGANISTAN', 'JAPON']
paises_oceania = ['AUSTRALIA', 'VANUATU']

In [39]:
col_loc_nacionalidad = df.columns.get_loc('NACIONALIDAD')

for i in range(len(df)):
    nac = df.iloc[i, col_loc_nacionalidad]
    if nac in paises_americanorte:
        df.iloc[i, col_loc_nacionalidad] = 'AMERICANORTE'
    elif nac in paises_americacentro:
        df.iloc[i, col_loc_nacionalidad] = 'AMERICACENTRO'
    elif nac in paises_americasur:
        df.iloc[i, col_loc_nacionalidad] = 'AMERICASUR'
    elif nac in paises_africa:
        df.iloc[i, col_loc_nacionalidad] = 'AFRICA'
    elif nac in paises_europa:
        df.iloc[i, col_loc_nacionalidad] = 'EUROPA'
    elif nac in paises_asia:
        df.iloc[i, col_loc_nacionalidad] = 'ASIA'
    elif nac in paises_oceania:
        df.iloc[i, col_loc_nacionalidad] = 'OCEANIA'

In [40]:
df['NACIONALIDAD'].value_counts()

COLOMBIA         35548
-                  556
AMERICASUR         164
EUROPA             149
AMERICANORTE        86
ASIA                25
OCEANIA             22
AMERICACENTRO       15
AFRICA               1
Name: NACIONALIDAD, dtype: int64

# DF DESPUES

In [41]:
df.head()

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,SITIO,ARMA,MOVILAGRESOR,MOVILVICTIMA,EDAD,SEXO,NACIONALIDAD,MARCA
0,ENERO,AMAZONAS,LETICIA (CT),DOMINGO,MADRUGADA,JARDIN,URBANA,CASAS DE HABITACION,CONTUNDENTES,A PIE,A PIE,ADULTOJOVEN,FEMENINO,COLOMBIA,SAMSUNG
1,ENERO,ANTIOQUIA,BELLO,DOMINGO,MOVILIZACIONVESPERTINA,CIUDADELA DEL NORTE,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,ADULTO,FEMENINO,COLOMBIA,SAMSUNG
2,ENERO,ANTIOQUIA,BELLO,DOMINGO,NOCHE,VDA. SAN FELIX,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,ADULTO,MASCULINO,COLOMBIA,LENOVO
3,ENERO,ANTIOQUIA,BELLO,DOMINGO,NOCHE,SANTA RITA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,MOTOCICLETA,A PIE,ADULTOJOVEN,MASCULINO,COLOMBIA,SAMSUNG
4,ENERO,ANTIOQUIA,BELLO,DOMINGO,MANHANA,VDA. SAN FELIX,RURAL,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,MOTOCICLETA,VEHICULO,ADULTO,MASCULINO,COLOMBIA,APPLE


In [42]:
df.columns

Index(['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'DIA', 'HORA', 'BARRIO', 'ZONA',
       'SITIO', 'ARMA', 'MOVILAGRESOR', 'MOVILVICTIMA', 'EDAD', 'SEXO',
       'NACIONALIDAD', 'MARCA'],
      dtype='object')

# Columnas pendientes por categorizar/generalizar/procesar

    MUNICIPIO
    BARRIO
    SITIO

# Pandas Profiling

In [43]:
pandas_profiling.ProfileReport(df)

Number of variables,16
Number of observations,36566
Total Missing (%),0.0%
Total size in memory,4.2 MiB
Average record size in memory,121.0 B
Numeric,1
Categorical,15
Date,0
Text (Unique),0
Rejected,0
Distinct count,9


# Exportar a csv

In [44]:
df.to_csv(ruta_salida, index=False, encoding='utf8' )